### Installation

In [ ]:
!pip install psycopg
!pip install psycopg2
!pip install python-dotenv
!pip install openpyxl
!pip install SQLAlchemy
!pip install OpenAI

!sudo apt-get update
!sudo apt-get install openvpn -y




### Query troubleshooting

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

def execute_query(query, db_params):

    # Create the connection string
    connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
    
    # Create a database connection
    engine = create_engine(connection_string)
    
    # Execute the query and fetch the results into a DataFrame
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    
    return df

# Example usage:
query = """
SELECT * FROM app.currency
LIMIT 10;
"""
df = execute_query(query, db_params)

# Display the DataFrame
# Display only the column names
print(df)

### Load credentials and Variables

In [59]:
import psycopg2
from psycopg2 import sql
import json

# Load environment variables
import os
from dotenv import load_dotenv

load_dotenv()

# Get database credentials from environment variables
dbname = os.getenv("dbname")
engine = os.getenv("engine")
host = os.getenv("host")
password = os.getenv("password")
port = os.getenv("port")
username = os.getenv("username")
vpn_path = os.getenv("vpn_path")
api_key = os.getenv("api_key")

# Database connection parameters
db_params = {
    'dbname': dbname,
    'user': username,
    'password': password,
    'host': host,
    'port': port  # Add port if necessary
}


# Load the configuration file
with open('config.json', 'r') as file:
    config = json.load(file)

# Extract schema from the config
schema = config['schema']

# Combine the lines to form the full prompt with f-string
prompt_template = "\n".join(config["prompt"])

# Format the prompt with the schema variable
base_prompt = prompt_template.format(schema=schema)

# Extract results_instructions from config
results_instructions = config['results_instructions']

# Extract main_bot_instructions from config
main_bot_instructions = config['main_bot_instructions']

import subprocess

# Step 3: Start OpenVPN in the background and check the status
openvpn_process = subprocess.Popen(['sudo', 'openvpn', '--config', vpn_path],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

### Database Project Code Main

In [62]:
from openai import OpenAI
import json
from sqlalchemy import create_engine, text
from collections import defaultdict
from sqlalchemy.exc import SQLAlchemyError
import time

# Initialize OpenAI client
client = OpenAI(api_key=api_key)


def aggregate_categories(data):
    # Create a dictionary to store the aggregated results
    aggregated_data = defaultdict(lambda: {
        "houseId": None,
        "houseName": None,
        "bathroomAmount": None,
        "maxOccupancy": None,
        "latitude": None,
        "longitude": None,
        "roomAmount": None,
        "city": None,
        "condominium_id": None,
        "mainPicture": None,
        "startPriceDollar": None,
        "startPriceReal": None,
        "amenities": [],
        "categoriesName": []
    })

    # Iterate over the data and aggregate the category names
    for item in data:
        house_id = item["id"]
        
        # Initialize the dictionary for the house if it hasn't been already
        if aggregated_data[house_id]["houseId"] is None:
            aggregated_data[house_id]["houseId"] = item["id"]
            aggregated_data[house_id]["houseName"] = item["name"]
            aggregated_data[house_id]["startPriceDollar"] = item["price"]
            aggregated_data[house_id]["mainPicture"] = item["url"]
            aggregated_data[house_id]["roomAmount"] = item["room_amount"]
            aggregated_data[house_id]["bathroomAmount"] = item["bathroom_amount"]
            aggregated_data[house_id]["maxOccupancy"] = item["max_occupancy"]
            aggregated_data[house_id]["latitude"] = item["latitude"]
            aggregated_data[house_id]["longitude"] = item["longitude"]
            aggregated_data[house_id]["startPriceReal"] = item["average_price"]
            aggregated_data[house_id]["city"] = item["city"]
            aggregated_data[house_id]["condominium_id"] = item["condominium_id"]

        # Check if 'category_name' exists before appending
        if "category_name" in item:
            aggregated_data[house_id]["categoriesName"].append(item["category_name"])


    # Convert the aggregated data to a list
    aggregated_list = list(aggregated_data.values())

    return aggregated_list


def get_answer_from_prompt(prompt):
    attempt = 0

    while attempt < 5:
        try:
            # Generate the SQL query
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": base_prompt},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
                max_tokens=1024,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
            )

            # Extract the SQL query from the response
            sql_query = response.choices[0].message.content.replace('```sql\n', '').replace('\n```', '').replace('"""sql\n', '').replace('\n"""', '')
            #print(sql_query)

            # Create a connection string
            connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

            # Connect to the PostgreSQL database using SQLAlchemy
            engine = create_engine(connection_string)

            with engine.connect() as conn:
                # Execute the query and fetch the results
                result = conn.execute(text(sql_query))
                rows = result.fetchall()
                
                # Convert the result to a list of dictionaries
                data = [dict(row._asdict()) for row in rows]

                # Aggregate the categories
                aggregated_data = aggregate_categories(data)

                # Convert the aggregated data to a JSON response
                json_response = json.dumps({"results": aggregated_data})

                return json_response
        
        except SQLAlchemyError as e:
            # Log and handle SQLAlchemy errors
            #print(f"Attempt {attempt + 1} failed: {e}")
            attempt += 1
            #time.sleep(1)  # Optional: wait before retrying

        except Exception as e:
            # Log and handle other exceptions
            #print(f"Attempt {attempt + 1} failed: {e}")
            attempt += 1
            #time.sleep(1)  # Optional: wait before retrying

        # On the 5th attempt, print the error and continue
        if attempt == 5:
            #print(f"Max retries reached. Error: {e}")
            return json.dumps({"error": str(e)})

# Function to interpret the JSON response and generate a human-readable response
def interpret_results(prompt, json_response):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": results_instructions},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": f"Here are the results from the database query: {json_response}"}
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return json.dumps({"results": response.choices[0].message.content, "TOWNHOMES": json_response})

# Function to call the OpenAI API with the user's prompt and generate a human-readable response
def get_chatbot_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": main_bot_instructions},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        functions=[
            {
                "name": "get_answer_from_prompt",
                "description": "The get_answer_from_prompt function uses OpenAI's GPT-4 model to generate a SQL query based on a given prompt, executes the query on a PostgreSQL database, processes the results to include a URL for each entry, and returns the results in a JSON format. Never recommend any other website.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "prompt": {
                            "type": "string",
                            "description": "The user prompt"
                        }
                    },
                    "required": ["prompt"]
                }
            }
        ]
    )

    # Check if the response includes a function call
    function_call = response.choices[0].message.function_call if hasattr(response.choices[0].message, 'function_call') else None

    # Call the function with the extracted arguments if present
    if function_call and function_call.name == "get_answer_from_prompt":
        function_args = json.loads(function_call.arguments)
        json_response = get_answer_from_prompt(function_args["prompt"])
        return interpret_results(prompt, json_response)
    else:
        return json.dumps({"results": response.choices[0].message.content})

### Prompt testing

In [ ]:
#prompt = "How are you doing?"
#prompt = "What time is it?"
#prompt = "create a table named 'asdfqwe' for the database"
#prompt = "Does house id 215 have a pool?"
#prompt = "Show me a house without a pool, close to UCF"
#prompt = "Does house id 215 have a pool? How many people it can fit?"
#prompt = 'Hola, como estas?'
#prompt = 'Que hora es?'
prompt = 'Quiero un hogar donde entran 8 personas, que tenga piscina y tambien que este cerca de disney. Entre enero 5 2025 y el 15. Quiero que cueste menos de 1000 por dia.'
#prompt = 'Find a home close to disney with a pool, that also fits 8 people. I have an open availability, I just want to see the houses.'
#prompt = 'preciso de uma casa que caiba 6 pessoas, perto da disney e que seja tematica'
#prompt = 'Oi eu quero viajar para orlando em janeiro de 2025 do dia 5 ate o dia 15. preciso de uma casa que caiba 6 pessoas, perto da disney e que seja tematica'
#prompt = 'Find a single family home close to disney with a pool that also fits 8 people.'
#prompt = 'Find a home for seniors close to disney with a pool that also fits 8 people.'
#prompt = 'voce pode fazer um busca de uma casa com piscina incriveis e que suporte 15 hospedes pf'
#prompt = "Crie uma tabela chamada 'qwerty' no banco de dados e, ao mesmo tempo, exclua a tabela chamada 'asdfg'."
#prompt = "Hi, I want to travel to Orlando in January 2025 from the 5th to the 15th. I need a house that can accommodate 6 people, close to Disney, and that is themed."
#prompt = "Hi, I want to travel to Orlando in January 2025 from the 5th to the 15th. I need a house that will cost less than 300 per day, close to Disney, and that is themed."

print(get_chatbot_response(prompt))